In [1]:
# Sample program for one scrip
from ib_insync import *
util.startLoop()
# ib=IB().connect('127.0.0.1', 7496, clientId=9) # kavi tws live
ib = IB().connect('127.0.0.1', 4001, clientId=9) # kavi IBG live

In [162]:
%%time
import numpy as np
import pandas as pd
from itertools import product
import datetime

import utils # for catch function

# sd multiple for band
sigma = 2       # 2 sigma is about 95% probability
penalty = 1.8   # e.g. 1.2 is 20% above

# market
exchange = 'NSE'

#... prepare lot dataframe for underlying
# from 5paisa
paisaurl = "https://www.5paisa.com/5pit/spma.asp"
df_paisa = pd.read_html(paisaurl, header=0)[1].drop_duplicates(subset='Symbol')

# Rename Symbol and Margin fields
df_paisa = df_paisa.rename(columns={'Symbol': 'nseSymbol', 'TotMgn%': 'marginpct'})

# Convert columns to numeric and make margin to pct
df_paisa = df_paisa.apply(pd.to_numeric, errors='ignore')
df_paisa.marginpct = df_paisa.marginpct.div(100)

# Truncate to 9 characters for ibSymbol
df_paisa['ibSymbol'] = df_paisa.nseSymbol.str.slice(0,9)

# nseSymbol to ibSymbol dictionary for conversion
ntoi = {'M&M': 'MM', 'M&MFIN': 'MM', 'L&TFH': 'LTFH', 'NIFTY': 'NIFTY50'}

# remap ibSymbol, based on the dictionary
df_paisa.ibSymbol = df_paisa.ibSymbol.replace(ntoi)

# %%time
#... Get the scrip
symbol = 'ACC'
contract = Stock('ACC', 'NSE')
ib.qualifyContracts(contract)

#... Get volatility, hi52 and lo52
duration = '12 M'
size = '1 day'
bars = ib.reqHistoricalData(contract=contract, endDateTime='', 
                     durationStr=duration, barSizeSetting=size, 
                     whatToShow='MIDPOINT', useRTH=True, 
                     formatDate=1, keepUpToDate=True)
stDev = np.std(a=[b.close for b in bars], ddof=0)

hi52 = max([b.high for b in bars])
lo52 = min([b.low for b in bars])

meanPrice = np.mean([b.close for b in bars])

#... Get the lot and margin
lot = df_paisa.loc[df_paisa.ibSymbol == symbol, 'Mlot'].item()
margin = df_paisa.loc[df_paisa.ibSymbol == symbol, 'TotMgnPerLt'].item()

#... Get the option chain tickers
chains = ib.reqSecDefOptParams(underlyingSymbol=contract.symbol, 
                      futFopExchange='', 
                      underlyingConId=contract.conId, underlyingSecType=contract.secType)

expiries = set(*[c.expirations for c in chains])

cds = [ib.reqContractDetails(Option(symbol, e, exchange='NSE')) for e in expiries]

options = [c.contract for cs in cds for c in cs]

tickers = [t for i in range(0, len(options), 100) for t in ib.reqTickers(*options[i:i + 100])]

# keep only those tickers with underlying prices
lib_t = {t: utils.catch(lambda: t.modelGreeks.undPrice) for t in tickers}
und_t = [k for k, v in lib_t.items() if v is not None]


#... Weed out unwanted SDs and make the option chain dataframe
# keep only the Ps and Cs outside the sigma band (95% probability)
tgt_ts = [t for t in und_t 
if ((t.contract.strike < t.modelGreeks.undPrice - stDev*sigma) & (t.contract.right == 'P')) | 
((t.contract.strike > t.modelGreeks.undPrice + stDev*sigma) & (t.contract.right == 'C'))]

# ... Build the dataframe
ts = [(t.contract.conId, t.contract.symbol, t.contract.lastTradeDateOrContractMonth, t.contract.strike, t.contract.right, t.modelGreeks.undPrice,
  t.contract.localSymbol, t.bid, t.bidSize, t.ask, t.askSize, t.close, t.modelGreeks.impliedVol, t.modelGreeks.delta, t.modelGreeks.optPrice, 
  t.modelGreeks.pvDividend, t.modelGreeks.gamma, t.modelGreeks.vega, t.modelGreeks.theta, t) for t in tgt_ts]

cols = ['conId', 'ibSymbol', 'expiry', 'strike', 'right', 'undPrice', 
        'localSymbol', 'bid', 'bidSize', 'ask', 'askSize', 'close', 'impliedVol', 'delta', 'optPrice', 
        'pvDividend', 'gamma', 'vega', 'theta', 'ticker']
df = pd.DataFrame(ts, columns=cols).sort_values(by=['expiry', 'strike'], ascending=False)

df['lot'] = lot
df['margin'] = margin
df['undHi'] = hi52
df['undLo'] = lo52
df['undMean'] = meanPrice
df['stDev'] = stDev

# Expected price is rounded up to 0.05 centrs after putting the penalty
df['expPrice'] = round(df[['bid', 'ask', 'close']].max(axis=1)*penalty * 2, 1)/2

df['dte'] = (pd.to_datetime(df.expiry) - datetime.datetime.now()).dt.days

df['rom'] = (df.expPrice*df.lot)/df.margin*252/df.dte

Wall time: 1min 8s


In [167]:
df.loc[:, ['strike', 'stDev', 'undLo', 'undHi', 'undPrice', 'undMean', 'right', 'expiry', 'bid', 'ask', 'close', 'expPrice', 'rom']].describe()

,strike,stDev,undLo,undHi,undPrice,undMean,bid,ask,close,expPrice,rom
count,47.000000,47.000000,4.700000e+01,4.700000e+01,47.00,4.700000e+01,47.0,47.0,47.000000,47.000000,47.000000
mean,1364.255319,138.543844,1.256225e+03,1.856975e+03,1537.25,1.517988e+03,-1.0,-1.0,0.203191,0.367021,0.012073
std,392.271782,0.000000,2.298318e-13,9.193274e-13,0.00,4.596637e-13,0.0,0.0,0.598949,1.078711,0.027242
min,940.000000,138.543844,1.256225e+03,1.856975e+03,1537.25,1.517988e+03,-1.0,-1.0,0.000000,0.000000,0.000000
25%,1050.000000,138.543844,1.256225e+03,1.856975e+03,1537.25,1.517988e+03,-1.0,-1.0,0.000000,0.000000,0.000000
50%,1180.000000,138.543844,1.256225e+03,1.856975e+03,1537.25,1.517988e+03,-1.0,-1.0,0.000000,0.000000,0.000000
75%,1850.000000,138.543844,1.256225e+03,1.856975e+03,1537.25,1.517988e+03,-1.0,-1.0,0.025000,0.050000,0.006760
max,1980.000000,138.543844,1.256225e+03,1.856975e+03,1537.25,1.517988e+03,-1.0,-1.0,3.050000,5.500000,0.135204


In [166]:
df = df.sort_values(by=['expiry', 'strike'], ascending=False)

In [169]:
df[df.rom > 0.13]

,conId,ibSymbol,expiry,strike,right,undPrice,localSymbol,bid,bidSize,ask,...,ticker,lot,margin,undHi,undLo,undMean,stDev,expPrice,dte,rom
38,339361800,ACC,20190131,1260.0,P,1537.25,ACC19JAN1260PE,-1.0,0,-1.0,...,"Ticker(contract=Option(conId=339361800, symbol...",400,107907.0,1856.975,1256.225,1517.987913,138.543844,5.5,38,0.135204


In [4]:
[print(str(s)) for s in ('ACC', 'INFY')]

ACC
INFY


[None, None]